In [73]:
import pandas as pd
import datetime 
import requests
from io import StringIO
from bs4 import BeautifulSoup
from lxml import html
import time

### URL para 0kms: https://autos.mercadolibre.com.ar/0-km/okm_Desde_1_NoIndex_True y URL para usados: https://autos.mercadolibre.com.ar/usados-autos_Desde_1_NoIndex_True

In [25]:
okm_url = "https://autos.mercadolibre.com.ar/0-km/okm_NoIndex_True#applied_filter_id%3DKILOMETERS%26applied_filter_name%3DKil%C3%B3metros%26applied_filter_order%3D4%26applied_value_id%3D%5B0km-0km%5D%26applied_value_name%3D0+km%26applied_value_order%3D2%26applied_value_results%3D531%26is_custom%3Dfalse"
usados_url = "https://autos.mercadolibre.com.ar/usados-autos_ITEM*CONDITION_2230581_NoIndex_True#applied_filter_id%3DITEM_CONDITION%26applied_filter_name%3DCondici%C3%B3n%26applied_filter_order%3D5%26applied_value_id%3D2230581%26applied_value_name%3DUsado%26applied_value_order%3D2%26applied_value_results%3D506%26is_custom%3Dfalse"

In [26]:
fecha_hora_actual = datetime.datetime.now()
# Formatear la fecha y hora en un formato específico
formato_personalizado = "%Y-%m-%d %H:%M:%S"  # Por ejemplo, "2023-08-25 15:30:00"
fecha_hora_formateada = fecha_hora_actual.strftime(formato_personalizado)
print("Fecha y hora:", fecha_hora_formateada)

Fecha y hora: 2023-09-01 22:21:26


### obtener las url de los 0kms

In [27]:
okm_links = []
i = 0
base_url = 'https://autos.mercadolibre.com.ar/0-km/okm_Desde_{page}_NoIndex_True'

while True:
    url = base_url.format(page=i + 1)
#     print(url)
    res = requests.get(url)
    
    if res.status_code != 200:
        break
        
    soup = BeautifulSoup(res.text, "html.parser")
    div_links_okm = soup.find_all('div', class_='ui-search-item__group ui-search-item__group--title shops__items-group')
    links_okm = [div.find('a')['href'] for div in div_links_okm]
    nombres_okm = [link.find('h2').get_text(strip=True) if link.find('h2') else '' for link in div_links_okm]
    precios_okm = soup.find_all('span', class_ = 'andes-money-amount__fraction')
    precios_okm = [precio.text.strip() for precio in precios_okm if precio != ""]
#     print({'precios_okm': precios_okm})
    
    for link, nom, prec in list(zip(links_okm, nombres_okm, precios_okm)):
#         print({'type': '0km', 'link_web': link, 'nom': nom})
        model_dict = {
            'type': '0km',
            'link_web': link,
            'modelo': nom,
            'precio': prec,
        }
        okm_links.append(model_dict)
    
    i += 48
# okm_links

In [28]:
def get_brand_model(row):
    modelo = row['modelo']
    modelo = modelo.split()
#     print(modelo)
    return modelo[0]

In [29]:
df_okm_model_links = pd.DataFrame(okm_links)
# df_okm_model_links['marca'] = df_okm_model_links.apply(get_brand_model, axis=1)
# df_okm_model_links

### obtener las url de los usados

In [30]:
usados_links = []
i = 0
base_url = 'https://autos.mercadolibre.com.ar/usados-autos_Desde_{page}_ITEM*CONDITION_2230581_NoIndex_True'

while True:
    url = base_url.format(page=i + 1)
#     print(url)
    res = requests.get(url)
    
    if res.status_code != 200:
        break
        
    soup = BeautifulSoup(res.text, "html.parser")
    div_links_usados = soup.find_all('div', class_='ui-search-item__group ui-search-item__group--title shops__items-group')
    links_usados = [div.find('a')['href'] for div in div_links_usados]
    nombres_usados = [link.find('h2').get_text(strip=True) if link.find('h2') else '' for link in div_links_usados]
    precios_usados = soup.find_all('span', class_ = 'andes-money-amount__fraction')
    precios_usados = [precio.text.strip() for precio in precios_usados if precio != ""]

    for link, nom, prec in list(zip(links_usados, nombres_usados, precios_usados)):
#         print({'type': 'usado', 'link_web': link, 'nom': nom})
        model_dict = {
            'type': 'usado',
            'link_web': link,
            'modelo': nom,
            'precio': prec,
        }
        usados_links.append(model_dict)
    
    i += 48
# usados_links

In [31]:
df_usados_model_links = pd.DataFrame(usados_links)
# df_usados_model_links

In [32]:
df_mercado_libre_links = pd.concat([df_okm_model_links, df_usados_model_links], ignore_index=True)
df_mercado_libre_links

,type,link_web,modelo,precio
0,0km,https://auto.mercadolibre.com.ar/MLA-145029973...,Chevrolet Tracker 1.2 Turbo At,8.099.852
1,0km,https://auto.mercadolibre.com.ar/MLA-148581689...,Peugeot 208 1.2 New Like,3.120.000
2,0km,https://auto.mercadolibre.com.ar/MLA-148582147...,Volkswagen Amarok 3.0 V6 Extreme,28.500.003
3,0km,https://auto.mercadolibre.com.ar/MLA-148579483...,Volkswagen Amarok 3.0 V6 Cd Highline,26.850.003
4,0km,https://auto.mercadolibre.com.ar/MLA-148584696...,Volkswagen Amarok 3.0 V6 Cd Comfortline,21.550.003
...,...,...,...,...
1053,usado,https://auto.mercadolibre.com.ar/MLA-138129962...,Peugeot 2008 1.6 Allure Tip,7.000.000
1054,usado,https://auto.mercadolibre.com.ar/MLA-144740810...,Jeep Renegade 1.8 Sport,12.490.000
1055,usado,https://auto.mercadolibre.com.ar/MLA-137762034...,Volkswagen Amarok 3.0 V6 Extreme,14.500.000
1056,usado,https://auto.mercadolibre.com.ar/MLA-146356845...,Kia Sportage 2.0 Ex 2wd 163cv At,7.655.000


In [33]:
df_mercado_libre_links.to_csv('../CSV/df_mercado_libre_model_links.csv', index=False)

In [34]:
df_mercado_libre_links = pd.read_csv('../CSV/df_mercado_libre_model_links.csv')
df_mercado_libre_links['marca'] = df_mercado_libre_links.apply(get_brand_model, axis=1)
# df_mercado_libre_links

In [35]:
df_mercado_libre_links[df_mercado_libre_links['type'] == '0km']

,type,link_web,modelo,precio,marca
0,0km,https://auto.mercadolibre.com.ar/MLA-145029973...,Chevrolet Tracker 1.2 Turbo At,8.099.852,Chevrolet
1,0km,https://auto.mercadolibre.com.ar/MLA-148581689...,Peugeot 208 1.2 New Like,3.120.000,Peugeot
2,0km,https://auto.mercadolibre.com.ar/MLA-148582147...,Volkswagen Amarok 3.0 V6 Extreme,28.500.003,Volkswagen
3,0km,https://auto.mercadolibre.com.ar/MLA-148579483...,Volkswagen Amarok 3.0 V6 Cd Highline,26.850.003,Volkswagen
4,0km,https://auto.mercadolibre.com.ar/MLA-148584696...,Volkswagen Amarok 3.0 V6 Cd Comfortline,21.550.003,Volkswagen
...,...,...,...,...,...
519,0km,https://auto.mercadolibre.com.ar/MLA-138196186...,Mercedes-benz Sprinter 316 Mixto 4+1 3665 Te,86.800,Mercedes-benz
520,0km,https://auto.mercadolibre.com.ar/MLA-145331258...,Chery Tiggo 4 Pro 1.5 Comfort Mt,14.900.000,Chery
521,0km,https://auto.mercadolibre.com.ar/MLA-144144665...,Toyota Hilux 2.8 Cd Srx 177cv 4x4 At,400.000,Toyota
522,0km,https://auto.mercadolibre.com.ar/MLA-925824993...,Foton Aumark Tm1 C/d,12.800.000,Foton


In [36]:
def get_models_specifications(row):
    url = row['link_web']
    specs = None
    res = requests.get(url)
    soup = BeautifulSoup(res.text, "html.parser")
    tree = html.fromstring(res.content)
    
    #modelo = soup.find('h1', class_='ui-pdp-title').text.strip()
    anio = soup.find('span', class_='ui-pdp-subtitle').text.strip()
    
    list_specs = soup.find_all('span', class_='andes-table__column--value')
    default_value = "Valor por defecto"

    if len(list_specs) != 0:
        specs = [span.text.strip() for span in list_specs]
        
#     print(specs)
    
    if specs and len(specs) != 0 and len(anio) != 0:
        return (
            anio[0:4], 
            specs[3] if len(specs) > 3 else "N/A", #VERSION
            specs[4] if len(specs) > 4 else "N/A", #COLOR
            specs[5] if len(specs) > 5 else "N/A", #COMBUSTIBLE
            specs[6] if len(specs) > 6 else "N/A", #PUERTAS
            specs[7] if len(specs) > 7 else "N/A", #TRANSMISION
            specs[8] if len(specs) > 8 else "N/A", #MOTOR
            specs[9] if len(specs) > 9 else "N/A" #VEHICULO
        )
    
    return ( 
        "N/A", 
        "N/A", #VERSION
        "N/A", #COLOR
        "N/A", #COMBUSTIBLE
        "N/A", #PUERTAS
        "N/A", #TRANSMISION
        "N/A", #MOTOR,
        "N/A" #VEHICULO
    )

In [37]:
# df_modelos_okm = pd.DataFrame(modelos_okm)
df_modelos_okm = df_mercado_libre_links[df_mercado_libre_links['type'] == '0km']
df_modelos_okm

,type,link_web,modelo,precio,marca
0,0km,https://auto.mercadolibre.com.ar/MLA-145029973...,Chevrolet Tracker 1.2 Turbo At,8.099.852,Chevrolet
1,0km,https://auto.mercadolibre.com.ar/MLA-148581689...,Peugeot 208 1.2 New Like,3.120.000,Peugeot
2,0km,https://auto.mercadolibre.com.ar/MLA-148582147...,Volkswagen Amarok 3.0 V6 Extreme,28.500.003,Volkswagen
3,0km,https://auto.mercadolibre.com.ar/MLA-148579483...,Volkswagen Amarok 3.0 V6 Cd Highline,26.850.003,Volkswagen
4,0km,https://auto.mercadolibre.com.ar/MLA-148584696...,Volkswagen Amarok 3.0 V6 Cd Comfortline,21.550.003,Volkswagen
...,...,...,...,...,...
519,0km,https://auto.mercadolibre.com.ar/MLA-138196186...,Mercedes-benz Sprinter 316 Mixto 4+1 3665 Te,86.800,Mercedes-benz
520,0km,https://auto.mercadolibre.com.ar/MLA-145331258...,Chery Tiggo 4 Pro 1.5 Comfort Mt,14.900.000,Chery
521,0km,https://auto.mercadolibre.com.ar/MLA-144144665...,Toyota Hilux 2.8 Cd Srx 177cv 4x4 At,400.000,Toyota
522,0km,https://auto.mercadolibre.com.ar/MLA-925824993...,Foton Aumark Tm1 C/d,12.800.000,Foton


In [38]:
df_modelos_okm['anio'], df_modelos_okm['version'], df_modelos_okm['color'], df_modelos_okm['combustible'], df_modelos_okm['puertas'] , df_modelos_okm['transmision'] , df_modelos_okm['motor'] , df_modelos_okm['vehiculo'] = zip(*df_modelos_okm.apply(get_models_specifications, axis=1))
# df_modelos_okm['MODELO_'], df_modelos_okm['anio'], df_modelos_okm['precio'] = zip(*df_modelos_okm.apply(get_models_okm_specifications, axis=1))

C:\Users\PC\AppData\Local\Temp\ipykernel_13348\4010890463.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_modelos_okm['anio'], df_modelos_okm['version'], df_modelos_okm['color'], df_modelos_okm['combustible'], df_modelos_okm['puertas'] , df_modelos_okm['transmision'] , df_modelos_okm['motor'] , df_modelos_okm['vehiculo'] = zip(*df_modelos_okm.apply(get_models_specifications, axis=1))
C:\Users\PC\AppData\Local\Temp\ipykernel_13348\4010890463.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_model

In [39]:
df_modelos_okm

,type,link_web,modelo,precio,marca,anio,version,color,combustible,puertas,transmision,motor,vehiculo
0,0km,https://auto.mercadolibre.com.ar/MLA-145029973...,Chevrolet Tracker 1.2 Turbo At,8.099.852,Chevrolet,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
1,0km,https://auto.mercadolibre.com.ar/MLA-148581689...,Peugeot 208 1.2 New Like,3.120.000,Peugeot,2023,1.2 New Like,Blanco,Nafta,5,Manual,1.2,Hatchback
2,0km,https://auto.mercadolibre.com.ar/MLA-148582147...,Volkswagen Amarok 3.0 V6 Extreme,28.500.003,Volkswagen,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
3,0km,https://auto.mercadolibre.com.ar/MLA-148579483...,Volkswagen Amarok 3.0 V6 Cd Highline,26.850.003,Volkswagen,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
4,0km,https://auto.mercadolibre.com.ar/MLA-148584696...,Volkswagen Amarok 3.0 V6 Cd Comfortline,21.550.003,Volkswagen,2023,3.0 V6 Cd Comfortline,Blanco,Diésel,4,Automática,3.0,Pick-Up
...,...,...,...,...,...,...,...,...,...,...,...,...,...
519,0km,https://auto.mercadolibre.com.ar/MLA-138196186...,Mercedes-benz Sprinter 316 Mixto 4+1 3665 Te,86.800,Mercedes-benz,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
520,0km,https://auto.mercadolibre.com.ar/MLA-145331258...,Chery Tiggo 4 Pro 1.5 Comfort Mt,14.900.000,Chery,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
521,0km,https://auto.mercadolibre.com.ar/MLA-144144665...,Toyota Hilux 2.8 Cd Srx 177cv 4x4 At,400.000,Toyota,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
522,0km,https://auto.mercadolibre.com.ar/MLA-925824993...,Foton Aumark Tm1 C/d,12.800.000,Foton,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A


In [40]:
df_modelos_usados = df_mercado_libre_links[df_mercado_libre_links['type'] == 'usado']
df_modelos_usados

,type,link_web,modelo,precio,marca
524,usado,https://auto.mercadolibre.com.ar/MLA-138603956...,Fiat Toro 2.0 Ranch 4x4 At,17.000.000,Fiat
525,usado,https://auto.mercadolibre.com.ar/MLA-149205120...,Volkswagen Polo 1.6 Msi Comfortline At,8.800.000,Volkswagen
526,usado,https://auto.mercadolibre.com.ar/MLA-138599359...,Ford Fiesta Kinetic Design 1.6 Se Powershift 1...,7.900.000,Ford
527,usado,https://auto.mercadolibre.com.ar/MLA-138592955...,Chevrolet Prisma 1.4 Ltz 98cv,8.900.000,Chevrolet
528,usado,https://auto.mercadolibre.com.ar/MLA-149117071...,Renault Sandero Stepway 1.6 Privilege 105cv,7.813.750,Renault
...,...,...,...,...,...
1053,usado,https://auto.mercadolibre.com.ar/MLA-138129962...,Peugeot 2008 1.6 Allure Tip,7.000.000,Peugeot
1054,usado,https://auto.mercadolibre.com.ar/MLA-144740810...,Jeep Renegade 1.8 Sport,12.490.000,Jeep
1055,usado,https://auto.mercadolibre.com.ar/MLA-137762034...,Volkswagen Amarok 3.0 V6 Extreme,14.500.000,Volkswagen
1056,usado,https://auto.mercadolibre.com.ar/MLA-146356845...,Kia Sportage 2.0 Ex 2wd 163cv At,7.655.000,Kia


In [41]:
df_modelos_usados['anio'], df_modelos_usados['version'], df_modelos_usados['color'], df_modelos_usados['combustible'], df_modelos_usados['puertas'] , df_modelos_usados['transmision'] , df_modelos_usados['motor'] , df_modelos_usados['vehiculo'] = zip(*df_modelos_usados.apply(get_models_specifications, axis=1))

C:\Users\PC\AppData\Local\Temp\ipykernel_13348\2198305565.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_modelos_usados['anio'], df_modelos_usados['version'], df_modelos_usados['color'], df_modelos_usados['combustible'], df_modelos_usados['puertas'] , df_modelos_usados['transmision'] , df_modelos_usados['motor'] , df_modelos_usados['vehiculo'] = zip(*df_modelos_usados.apply(get_models_specifications, axis=1))
C:\Users\PC\AppData\Local\Temp\ipykernel_13348\2198305565.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vi

In [42]:
df_modelos_usados

,type,link_web,modelo,precio,marca,anio,version,color,combustible,puertas,transmision,motor,vehiculo
524,usado,https://auto.mercadolibre.com.ar/MLA-138603956...,Fiat Toro 2.0 Ranch 4x4 At,17.000.000,Fiat,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
525,usado,https://auto.mercadolibre.com.ar/MLA-149205120...,Volkswagen Polo 1.6 Msi Comfortline At,8.800.000,Volkswagen,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
526,usado,https://auto.mercadolibre.com.ar/MLA-138599359...,Ford Fiesta Kinetic Design 1.6 Se Powershift 1...,7.900.000,Ford,2016,1.6 Se Powershift 120cv,Blanco,Nafta,5,Automática,1.6,Hatchback
527,usado,https://auto.mercadolibre.com.ar/MLA-138592955...,Chevrolet Prisma 1.4 Ltz 98cv,8.900.000,Chevrolet,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
528,usado,https://auto.mercadolibre.com.ar/MLA-149117071...,Renault Sandero Stepway 1.6 Privilege 105cv,7.813.750,Renault,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1053,usado,https://auto.mercadolibre.com.ar/MLA-138129962...,Peugeot 2008 1.6 Allure Tip,7.000.000,Peugeot,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
1054,usado,https://auto.mercadolibre.com.ar/MLA-144740810...,Jeep Renegade 1.8 Sport,12.490.000,Jeep,2023,1.8 Sport,Negro,Nafta,5,Manual,1.8,SUV
1055,usado,https://auto.mercadolibre.com.ar/MLA-137762034...,Volkswagen Amarok 3.0 V6 Extreme,14.500.000,Volkswagen,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
1056,usado,https://auto.mercadolibre.com.ar/MLA-146356845...,Kia Sportage 2.0 Ex 2wd 163cv At,7.655.000,Kia,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A


In [43]:
df_mercado_libre_modelos = pd.concat([df_modelos_okm, df_modelos_usados], ignore_index=True)
df_mercado_libre_modelos.to_csv('../CSV/df_mercado_libre_modelos.csv', index=False)
df_mercado_libre_modelos

,type,link_web,modelo,precio,marca,anio,version,color,combustible,puertas,transmision,motor,vehiculo
0,0km,https://auto.mercadolibre.com.ar/MLA-145029973...,Chevrolet Tracker 1.2 Turbo At,8.099.852,Chevrolet,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
1,0km,https://auto.mercadolibre.com.ar/MLA-148581689...,Peugeot 208 1.2 New Like,3.120.000,Peugeot,2023,1.2 New Like,Blanco,Nafta,5,Manual,1.2,Hatchback
2,0km,https://auto.mercadolibre.com.ar/MLA-148582147...,Volkswagen Amarok 3.0 V6 Extreme,28.500.003,Volkswagen,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
3,0km,https://auto.mercadolibre.com.ar/MLA-148579483...,Volkswagen Amarok 3.0 V6 Cd Highline,26.850.003,Volkswagen,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
4,0km,https://auto.mercadolibre.com.ar/MLA-148584696...,Volkswagen Amarok 3.0 V6 Cd Comfortline,21.550.003,Volkswagen,2023,3.0 V6 Cd Comfortline,Blanco,Diésel,4,Automática,3.0,Pick-Up
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1053,usado,https://auto.mercadolibre.com.ar/MLA-138129962...,Peugeot 2008 1.6 Allure Tip,7.000.000,Peugeot,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
1054,usado,https://auto.mercadolibre.com.ar/MLA-144740810...,Jeep Renegade 1.8 Sport,12.490.000,Jeep,2023,1.8 Sport,Negro,Nafta,5,Manual,1.8,SUV
1055,usado,https://auto.mercadolibre.com.ar/MLA-137762034...,Volkswagen Amarok 3.0 V6 Extreme,14.500.000,Volkswagen,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
1056,usado,https://auto.mercadolibre.com.ar/MLA-146356845...,Kia Sportage 2.0 Ex 2wd 163cv At,7.655.000,Kia,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A


In [44]:
fecha_hora_actual = datetime.datetime.now()
# Formatear la fecha y hora en un formato específico
formato_personalizado = "%Y-%m-%d %H:%M:%S"  # Por ejemplo, "2023-08-25 15:30:00"
fecha_hora_formateada = fecha_hora_actual.strftime(formato_personalizado)
print("Fecha y hora:", fecha_hora_formateada)

Fecha y hora: 2023-09-01 22:40:16


### obtenemos datos de los 0km de https://autocity.com.ar/catalogo/0km/ para ampliar la data

In [76]:
okm_links_autocity = []
base_url = 'https://autocity.com.ar/catalogo/0km/page/{page}/'
page = 1

while True:
    url = base_url.format(page=page)
    res = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
    
    if res.status_code != 200:
        break

    soup = BeautifulSoup(res.text, "html.parser")

    divs_nombre_okm_autocity = [div.find('h3').text.strip() for div in soup.find_all('div', class_='carousel-autocity-card-title')]
    links_models_autocity = [a['href'] for a in soup.select('div > ul > li > a') if 'catalogo/0km/' in a['href'] and len(a['href']) > 40 and 'page' not in a['href']]    
    if divs_nombre_okm_autocity and links_models_autocity:
        for nom, link in zip(divs_nombre_okm_autocity, links_models_autocity):
            model_dict = {
                'type': '0km',
#                 'modelo': nom,
                'link_web': link
            }
#             print(model_dict)
            okm_links_autocity.append(model_dict)

        page += 1
        time.sleep(2)  # Agrega una pausa entre solicitudes
    else:
        break

print(okm_links_autocity)

[{'type': '0km', 'link_web': 'https://autocity.com.ar/catalogo/0km/renault/kangoo/express-1-6-confort-sce-l18/'}, {'type': '0km', 'link_web': 'https://autocity.com.ar/catalogo/0km/nissan/frontier/2-3-d-4x2-cd-platinum-at-l22/'}, {'type': '0km', 'link_web': 'https://autocity.com.ar/catalogo/0km/volkswagen/amarok/2-0-td-180hp-4x2-dc-comfortline-l17/'}, {'type': '0km', 'link_web': 'https://autocity.com.ar/catalogo/0km/peugeot/208/1-6-feline-tiptonic-l20/'}, {'type': '0km', 'link_web': 'https://autocity.com.ar/catalogo/0km/fiat/pulse/1-0-impetus/'}, {'type': '0km', 'link_web': 'https://autocity.com.ar/catalogo/0km/chrysler/jeep-compass/1-3t-270-sport-at6/'}, {'type': '0km', 'link_web': 'https://autocity.com.ar/catalogo/0km/peugeot/partner/1-6-confort-l17/'}, {'type': '0km', 'link_web': 'https://autocity.com.ar/catalogo/0km/peugeot/208/1-2-new-like-l23/'}, {'type': '0km', 'link_web': 'https://autocity.com.ar/catalogo/0km/peugeot/208/1-6-like-pack-l20/'}, {'type': '0km', 'link_web': 'https:/

In [77]:
df_okm_links_autocity = pd.DataFrame(okm_links_autocity)
df_okm_links_autocity

,type,link_web
0,0km,https://autocity.com.ar/catalogo/0km/renault/k...
1,0km,https://autocity.com.ar/catalogo/0km/nissan/fr...
2,0km,https://autocity.com.ar/catalogo/0km/volkswage...
3,0km,https://autocity.com.ar/catalogo/0km/peugeot/2...
4,0km,https://autocity.com.ar/catalogo/0km/fiat/puls...
...,...,...
216,0km,https://autocity.com.ar/catalogo/0km/renault/k...
217,0km,https://autocity.com.ar/catalogo/0km/nissan/fr...
218,0km,https://autocity.com.ar/catalogo/0km/volkswage...
219,0km,https://autocity.com.ar/catalogo/0km/peugeot/2...


In [142]:
def get_more_car_specs(row):
    modelo = row['modelo'].lower()
    modelo = modelo.split()
    urls = list(df_okm_links_autocity['link_web'])
    
    filtered_urls = [{'modelo': modelo, 'url': url} for url in urls
                     if all(word in url for word in modelo[:2])]
    
    for item in filtered_urls:
        print(item)

In [143]:
df_mercado_libre_modelos = pd.read_csv('../CSV/df_mercado_libre_modelos.csv')
df_mercado_libre_modelos = df_mercado_libre_modelos[df_mercado_libre_modelos['type'] == '0km']

In [144]:
df_mercado_libre_modelos.apply(get_more_car_specs, axis=1)

{'modelo': ['peugeot', '208', '1.2', 'new', 'like'], 'url': 'https://autocity.com.ar/catalogo/0km/peugeot/208/1-6-feline-tiptonic-l20/'}
{'modelo': ['peugeot', '208', '1.2', 'new', 'like'], 'url': 'https://autocity.com.ar/catalogo/0km/peugeot/208/1-2-new-like-l23/'}
{'modelo': ['peugeot', '208', '1.2', 'new', 'like'], 'url': 'https://autocity.com.ar/catalogo/0km/peugeot/208/1-6-like-pack-l20/'}
{'modelo': ['peugeot', '208', '1.2', 'new', 'like'], 'url': 'https://autocity.com.ar/catalogo/0km/peugeot/208/1-2-like-pack-l23/'}
{'modelo': ['peugeot', '208', '1.2', 'new', 'like'], 'url': 'https://autocity.com.ar/catalogo/0km/peugeot/208/1-6-feline-tiptonic-l20/'}
{'modelo': ['peugeot', '208', '1.2', 'new', 'like'], 'url': 'https://autocity.com.ar/catalogo/0km/peugeot/208/1-6-active-l20/'}
{'modelo': ['peugeot', '208', '1.2', 'new', 'like'], 'url': 'https://autocity.com.ar/catalogo/0km/peugeot/208/1-6-feline-tiptonic-l20/'}
{'modelo': ['peugeot', '208', '1.2', 'new', 'like'], 'url': 'https://

0      None
1      None
2      None
3      None
4      None
       ... 
519    None
520    None
521    None
522    None
523    None
Length: 524, dtype: object

### obtenemos datos de los usados de https://autocity.com.ar/catalogo/usados/ para ampliar la data